Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import ensemble
from datetime import datetime
import scipy.stats as stats
import altair as alt
from vega_datasets import data

import clean csv in dataframes

In [ ]:
df_ventes = pd.read_csv(r"C:\Users\utilisateur\Documents\Projets\brief_librairie\ventes.csv")
df_produits = pd.read_csv(r"C:\Users\utilisateur\Documents\Projets\brief_librairie\produits.csv")
df_clients = pd.read_csv(r"C:\Users\utilisateur\Documents\Projets\brief_librairie\clients.csv")

Create a unique dataframe

In [ ]:
df_all = df_ventes.merge(df_clients, how='left', on='client_id').merge(df_produits, how='left', on='id_prod')
df_all

let's print the infos number of price

In [ ]:
df_all["price"].describe(include='all')

print where the price is equal to -1

In [ ]:
df_all.loc[df_all['price'] == -1]

We see that this datas are test data so we chose to delete them

In [ ]:
df_all = df_all[df_all.price != -1]
df_all

we have now 336816 columns cause we droped the 200 where the price was 1

let's check if the date of birth is legit

In [ ]:
df_all["birth"].describe(include='all')

we got a min at 1929 and a max at 2004 so the data seems legit

let's create a new columns to print the total of purchase by client id

In [ ]:
df_all = df_all.merge(
    df_all.pivot_table(
    index='client_id', values='price', 
    aggfunc='sum').reset_index().rename(
    columns={'price': 'total_achats'}),
    on='client_id', how='left')

let's see the 10 biggest clients

In [ ]:
df_10_clients = df_all.drop_duplicates(subset =["client_id"])
df_top_10 = df_10_clients.nlargest(10,"total_achats")
df_top_10

We can see a huge difference between the top 4 clients and the rest of the top 10 clients

In [ ]:
df_drop_client = df_all.drop_duplicates(subset=['client_id'])

calcul of the total revenue

In [ ]:
df_drop_client['total_achats'].sum()

The total revenue is 5,79 Millions

Calcul of the total revenue of the 4 biggest clients

In [ ]:
liste_top_10 = df_top_10["total_achats"].iloc[0:4]
liste_top_10.sum()

The total revenue of the 4 biggest clients is 433 000

Calcul of the pourcentage the 4 biggest clients represents

In [ ]:
(df_drop_client['total_achats'].sum())/(liste_top_10.sum())

The 4 biggest clients represents 13,3% of the total revenue

check the correlation of sex and category
to see this we need to transform the sex to a numerical value
0 for male 1 for female

In [ ]:
def clean(x):
    x = x.replace("f", "1").replace("m", "0")
    return (x)

df_all['sex'] = df_all['sex'].apply(clean)



now let's check the correlation bewtween sex and category

In [ ]:

df_sex_categ = df_all.groupby(by=["sex","categ"]).count().reset_index()
df_sex_categ

We can say that the categorie and the sex are not correlate

Let's see if the category depend of the price with a boxplot

In [ ]:
boxplot = df_all.boxplot(column=['price'], by=["categ"])

with this boxplot we can see that the category is influenced by the price like this :
price less that 50 category 0
price less than 100 category 1 
price more than 100 category 2

with this value we can confirm there is no correletion between the sex and the category
in fact the sex got no incidence with the value of an object

In [ ]:
dep = df_all.price
n = len(dep)
lorenz = np.cumsum(np.sort(dep)) / dep.sum()
lorenz = np.append([0],lorenz)

xaxis = np.linspace(0-1/n,1+1/n,n+1)
plt.plot([0,1], [0,1], color='k')
plt.plot(xaxis,lorenz)
plt.title("Courbe de lorenz des prix")
plt.show()

transform the birth date into ages

In [ ]:
def age(x):
    x = 2022-x
    return x
df_all = df_all.rename(columns={"birth": "age"})
df_all['age'] = df_all['age'].apply(age)
df_all = df_all.rename(columns={"birth": "age"})
df_all

Create new column with date with the format Y

In [ ]:
df_all['date_format'] = df_all['date'].str[0:7]
df_all

groupby the date and the age to see how many sail per month

In [ ]:
df_sail_month = df_all.groupby(by=["date_format","age"]).count().reset_index()
df_sail_month

marbark age by sails per month

In [ ]:
source = df_sail_month

alt.Chart(source).mark_bar().encode(
    alt.X("age", bin=alt.Bin(maxbins=20), title = "age"),
    alt.Y('total_achats', title = "sails per month"),
)

we can see that the clients that buy the most per month are the 40-45 years old

markbar sex by categ to visualise the correlation between the gender and the category of the purchase

changing category format from 0.1.2 to 1.2.3 because altair can't represent the 0

In [ ]:
def categ(x):
    x = x+1
    return(x)
df_all['categ'] = df_all["categ"].apply(categ)
df_all

In [ ]:
alt.data_transformers.disable_max_rows()
source = df_all

 alt.Chart(source).mark_bar().encode(
     x='sum(categ)',
     y='sex',
     color='categ',
     order=alt.Order(
       'categ',
       sort='ascending'
     )
 )

groupby age and category the correlation between those two

In [ ]:
source = df_all

 alt.Chart(source).mark_bar().encode(
     alt.X("sum(categ)", title = "category"),
     alt.Y('age', bin=alt.Bin(maxbins=10), title = "age"),
     color='categ',
     order=alt.Order(
       # Sort the segments of the bars by this field
       'categ',
       sort='ascending'
     )
)

with this graph, we can  see that the age of the client got a big influence of the category he buy.
The 20-30 buy the most product in the category 2 and 3
the 30-60 buy the most product in the category 1

In [ ]:
df_panier = df_all.groupby(["session_id","age"])['id_prod'].count().reset_index(name='id_prod')
df_panier = df_panier.groupby("age")['id_prod'].mean().reset_index(name='id_prod')
df_panier